# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
**Mid-term exam**

**Date**: 3/12/2025

**Student Name**: Ferdinand Josef Bierbaum Aguilar

**Professor**: Pablo Camarillo Ramirez

In [28]:
import findspark
findspark.init()

In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-FERDINAND-BIERBAUM") \
    .master("spark://2e9c357ab096:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

In [30]:
# products

from grandeInformacion.spark_utils import SparkUtils
products_schema = SparkUtils.generate_schema([("product_id", "integer"), ("product_name", "string"), ("category", "string"), ("price", "double")])

products_df = spark.read \
                .schema(products_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914N/products.csv")

products_df.printSchema()

# products_df.show(5, truncate=False)

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)



In [31]:
# sales

from grandeInformacion.spark_utils import SparkUtils
sales_schema = SparkUtils.generate_schema([("transaction_id", "integer"), ("product_id", "integer"), ("customer_id", "integer"), ("sale_amount", "double"), ("sale_date", "date")])

sales_df = spark.read \
                .schema(sales_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914N/sales.csv")

sales_df.printSchema()

# sales_df.show(5, truncate=False)

root
 |-- transaction_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- sale_amount: double (nullable = true)
 |-- sale_date: date (nullable = true)



In [32]:
# join

join_productId_df = products_df.join(sales_df, products_df["product_id"] == sales_df["product_id"], "inner")

# join_productId_df.show(5)

In [33]:
# group and aggregate
from pyspark.sql.functions import sum, when

grouped_df = join_productId_df.groupBy("category").agg(sum("sale_amount").alias("Total Amount of Category"))

# grouped_df.show(5)

In [34]:
# transformation

join_productId_df = join_productId_df.withColumn('sale_type', when(join_productId_df['sale_amount'] >= 50, 'High').otherwise("Low"))

# join_productId_df.show(5)


In [35]:
# filter
filtered_joined_df = join_productId_df.filter(join_productId_df["category"] == "Electronics")
# filtered_joined_df.show(5)

In [36]:
# select 

selected_columns_df = join_productId_df.select("product_name", "sale_amount", "sale_type")

# selected_columns_df.show(5)

In [37]:
# screenshot in the folder 


In [38]:
# actions
selected_columns_df.show(5)
count_total_rows = selected_columns_df.count()
print("Total rows: ", count_total_rows)

+------------+-----------+---------+
|product_name|sale_amount|sale_type|
+------------+-----------+---------+
| Product 188|     155.46|     High|
| Product 189|     292.97|     High|
| Product 180|      843.5|     High|
| Product 179|     486.38|     High|
| Product 165|     220.58|     High|
+------------+-----------+---------+
only showing top 5 rows

Total rows:  3797


In [39]:
# sc.stop()